# Open Archive

Open a file/object archive into a target directory

### Parameters

* **`context`**: `mlrun.MLClientCtx` - function execution context
* **`archive_url`**: `str` - url of archive file
* **`subdir`**: `str` - path within artifact store where extracted files are stored
* **`key`**: `str` - key of archive contents in artifact store
* **`target_path`**: `str` - file system path to store extracted files

### Usage

In [1]:
import mlrun
import os

project_name = "function-marketplace"
project = mlrun.get_or_create_project(project_name, context="./", user_project=False)

> 2022-12-01 13:17:49,210 [info] Created and saved project function-marketplace: {'from_template': None, 'overwrite': False, 'context': './', 'save': True}
> 2022-12-01 13:17:49,211 [info] created project function-marketplace and saved in MLRun DB


In [3]:
# Declaring our zip target url
ARCHIVE_URL = "https://s3.wasabisys.com/iguazio/data/cats-vs-dogs/cats-vs-dogs-labeling-demo.zip"

if os.environ.get("MLRUN_ARTIFACT_PATH"):# When custom artifact path is declared
    TARGET_PATH = os.environ["MLRUN_ARTIFACT_PATH"] + "Dataset/"
else:
    TARGET_PATH = os.getcwd()+"/Dataset/" # save local

# Import, mount & run
fn = mlrun.import_function("hub://open_archive")

# Mounting
fn.apply(mlrun.auto_mount())
if os.getenv('MLRUN_CE','False')=='true':
    fn.spec.disable_auto_mount=False
    
run = fn.run(inputs={'archive_url': ARCHIVE_URL},
             params={'key': 'test_archive'})

> 2022-12-01 13:21:05,585 [info] starting run open-archive-open_archive uid=11eff72e8abc4844a58ada18533d97cb DB=http://mlrun-api:8080
> 2022-12-01 13:21:05,801 [info] Job is running in the background, pod: open-archive-open-archive-nb75p
> 2022-12-01 13:21:13,223 [info] downloading https://s3.wasabisys.com/iguazio/data/cats-vs-dogs/cats-vs-dogs-labeling-demo.zip to local temp file
> 2022-12-01 13:22:33,153 [info] Using v3io scheme, extracting to /v3io/projects/function-marketplace/artifacts/content/
> 2022-12-01 13:22:41,176 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...533d97cb,0,Dec 01 13:21:13,completed,open-archive-open_archive,v3io_user=danikind=jobowner=danimlrun/client_version=1.2.0-rc22host=open-archive-open-archive-nb75p,archive_url,key=test_archive,,test_archive


> 2022-12-01 13:22:43,950 [info] run executed, status=completed


### Inspecting outputs

In [ ]:
print("Artifact URL : ", run.artifact('test_archive').url)

# getting folder size
size = 0
# get size
if not os.environ.get("MLRUN_ARTIFACT_PATH"):
    for path, dirs, files in os.walk(TARGET_PATH):
        for f in files:
            fp = os.path.join(path, f)
            size += os.stat(fp).st_size
            
    print("Extracted folder size : ", size)
    
    # Deleting folder
    import shutil
    shutil.rmtree(TARGET_PATH + '/data')